# Basismodels tests

In [1]:
import QuantLib as ql

# Testing deterministic tenor basis model with continuous compounded spreads...

In [2]:
terms = [ql.Period(0, ql.Days),   ql.Period(1, ql.Years), ql.Period(2, ql.Years),  ql.Period(3, ql.Years),
        ql.Period(5, ql.Years),  ql.Period(7, ql.Years), ql.Period(10, ql.Years), ql.Period(15, ql.Years),
        ql.Period(20, ql.Years), ql.Period(61, ql.Years)]

In [3]:
discRates = [-0.00147407, -0.001761684, -0.001736745, -0.00119244, 0.000896055,
             0.003537077, 0.007213824,  0.011391278,  0.013334611, 0.013982809]

In [4]:
proj3mRates = [-0.000483439, -0.000578569, -0.000383832, 0.000272656, 0.002478699,
               0.005100113,  0.008750643,  0.012788095,  0.014534052, 0.014942896]

In [5]:
proj6mRates = [0.000233608, 0.000218862, 0.000504018, 0.001240556, 0.003554415,
               0.006153921, 0.009688264, 0.013521628, 0.015136391, 0.015377704]

In [6]:
capletTerms = [ql.Period(1, ql.Years), ql.Period(2, ql.Years),  ql.Period(3, ql.Years),
        ql.Period(5, ql.Years),  ql.Period(7, ql.Years), ql.Period(10, ql.Years), ql.Period(15, ql.Years),
        ql.Period(20, ql.Years), ql.Period(25, ql.Years), ql.Period(30, ql.Years)]

In [7]:
capletStrikes = [-0.0050, 0.0000, 0.0050, 0.0100, 0.0150, 0.0200, 0.0300, 0.0500]

In [8]:
cplRow01 = [0.003010094, 0.002628065, 0.00456118,  0.006731268,
            0.008678572, 0.010570881, 0.014149552, 0.021000638]
cplRow02 = [0.004173715, 0.003727039, 0.004180263, 0.005726083,
            0.006905876, 0.008263514, 0.010555395, 0.014976523]
cplRow03 = [0.005870143, 0.005334526, 0.005599775, 0.006633987,
            0.007773317, 0.009036581, 0.011474391, 0.016277549]
cplRow04 = [0.007458597, 0.007207522, 0.007263995, 0.007308727,
            0.007813586, 0.008274858, 0.009743988, 0.012555171]
cplRow05 = [0.007711531, 0.007608826, 0.007572816, 0.007684107,
            0.007971932, 0.008283118, 0.009268828, 0.011574083]
cplRow06 = [0.007619605, 0.007639059, 0.007719825, 0.007823373,
            0.00800813,  0.008113384, 0.008616374, 0.009785436]
cplRow07 = [0.007312199, 0.007352993, 0.007369116, 0.007468333,
            0.007515657, 0.00767695,  0.008020447, 0.009072769]
cplRow08 = [0.006905851, 0.006966315, 0.007056413, 0.007116494,
            0.007259661, 0.00733308,  0.007667563, 0.008419696]
cplRow09 = [0.006529553, 0.006630731, 0.006749022, 0.006858027,
            0.007001959, 0.007139097, 0.007390404, 0.008036255]
cplRow10 = [0.006225482, 0.006404012, 0.00651594,  0.006642273,
            0.006640887, 0.006885713, 0.007093024, 0.00767373]

In [9]:
def getYTS(terms, rates, spread = 0.0):
    today = ql.Settings.instance().evaluationDate
    dates = [ql.NullCalendar().advance(today, term, ql.Unadjusted) for term in terms]
    ratesPlusSpread = [rate + spread for rate in rates]
    ts = ql.CubicZeroCurve(dates, ratesPlusSpread, ql.Actual365Fixed(), ql.NullCalendar())
    return ql.RelinkableYieldTermStructureHandle(ts)

In [10]:
def testSwaptioncfs(contTenorSpread : bool):
    # market data and floating rate index
    discYTS = getYTS(terms, discRates)
    proj6mYTS = getYTS(terms, proj6mRates)
    euribor6m = ql.Euribor6M(proj6mYTS)
    
    #Vanilla swap details
    today = ql.Settings.instance().evaluationDate
    swapStart = ql.TARGET().advance(today, ql.Period(5, ql.Years), ql.Following)
    swapEnd = ql.TARGET().advance(swapStart, ql.Period(10, ql.Years), ql.Following)
    exerciseDate = ql.TARGET().advance(swapStart, ql.Period(-2, ql.Days), ql.Preceding)
    fixedSchedule = ql.Schedule(swapStart, swapEnd, ql.Period(1, ql.Years), ql.TARGET(), ql.ModifiedFollowing,
                               ql.ModifiedFollowing, ql.DateGeneration.Backward, False)
    floatSchedule = ql.Schedule(swapStart, swapEnd, ql.Period(6, ql.Months), ql.TARGET(), ql.ModifiedFollowing,
                               ql.ModifiedFollowing, ql.DateGeneration.Backward, False)
    swap = ql.VanillaSwap(ql.VanillaSwap.Payer, 10000.0, fixedSchedule, 0.03, ql.Thirty360(),
                            floatSchedule, euribor6m, 0.0, euribor6m.dayCounter())
    swap.setPricingEngine(ql.DiscountingSwapEngine(discYTS))
    
    # European exercise and swaption
    europeanExercise = ql.EuropeanExercise(exerciseDate)
    swaption = ql.Swaption(swap, europeanExercise, ql.Settlement.Physical)
    
    # calculate basis model swaption cash flows, discount and conmpare with swap
    cashFlows = ql.SwaptionCashFlows(swaption, discYTS, contTenorSpread)
    
    # model time is always Act365Fixed
    exerciseTime = ql.Actual365Fixed().yearFraction(discYTS.referenceDate(),
                                                    swaption.exercise().dates()[0])
    
    if (exerciseTime != cashFlows.exerciseTimes()[0]):
        print("Error: Swaption cash flow exercise time does not coincide with manual calculation")
        
    tol = 1.0e-8
    
    # (discounted) fixed leg coupons must match swap fixed leg NPV
    fixedLeg = 0.0
    for k in range(0, len(cashFlows.fixedTimes())):
        fixedLeg += cashFlows.fixedWeights()[k] * discYTS.discount(cashFlows.fixedTimes()[k])
    
    if (abs(fixedLeg - (-swap.fixedLegNPV())) > tol): # note, '-1' because payer swap
        print("Error: Swaption cash flow fixed leg NPV does not match Vanillaswap fixed leg NPV")
    
    # (discounted) floating leg coupons must match swap floating leg NPV
    floatLeg = 0.0
    for k in range(0, len(cashFlows.floatTimes())):
        floatLeg += cashFlows.floatWeights()[k] * discYTS.discount(cashFlows.floatTimes()[k])
    
    if (abs(floatLeg - swap.floatingLegNPV()) > tol):
        print("Error: Swaption cash flow floating leg NPV does not match Vanillaswap floating leg NPV.")
        
    # There should not be spread coupons in a single-curve setting.
    # However, if indexed coupons are used the floating leg is not at par,
    # so we need to relax the tolerance to a level at which it will only
    # catch large errors.
    
    tol2 = tol
    singleCurveCashFlows = ql.SwaptionCashFlows(swaption, proj6mYTS, contTenorSpread)
    for k in range(1, len(cashFlows.fixedTimes()) - 1):
        if (abs(singleCurveCashFlows.floatWeights()[k]) > tol2):
            print("Error: Swaption cash flow floating leg spread does not vanish in single-curve setting.")

In [11]:
testSwaptioncfs(True)

In [12]:
testSwaptioncfs(False)

# Testing volatility transformation for caplets/floorlets...

In [13]:
spread = 0.01
discYTS = getYTS(terms, discRates)
proj3mYTS = getYTS(terms, proj3mRates)
proj6mYTS = getYTS(terms, proj3mRates, spread)
euribor3m = ql.Euribor3M(proj3mYTS)
euribor6m = ql.Euribor6M(proj6mYTS)

In [14]:
def getOptionletTS():
    today = ql.Settings.instance().evaluationDate
    dates = [ql.TARGET().advance(today, capletTerm, ql.Following) for capletTerm in capletTerms]
    capletVols = [cplRow01, cplRow02, cplRow03, cplRow04, cplRow05, cplRow06, cplRow07, cplRow08, cplRow09, cplRow10]
    capletVolQuotes = [[ql.RelinkableQuoteHandle(ql.SimpleQuote(capletVol)) for capletVol in capletVolRow] for capletVolRow in capletVols]
    curve3m = getYTS(terms, proj3mRates)
    index = ql.Euribor3M(curve3m)
    tmp1 = ql.StrippedOptionlet(2, ql.TARGET(), ql.Following, index, dates, capletStrikes,
                                  capletVolQuotes, ql.Actual365Fixed(), ql.Normal, 0.0)
    tmp2 = ql.StrippedOptionletAdapter(tmp1)
    return ql.RelinkableOptionletVolatilityStructureHandle(tmp2)

In [15]:
# 3m optionlet VTS
optionletVTS3m = getOptionletTS()

In [16]:
corrTimes = [0.0, 50.0]
rhoInfData = [0.3, 0.3]
betaData = [0.9, 0.9]

In [17]:
rho = ql.LinearInterpolation(corrTimes, rhoInfData)
beta = ql.LinearInterpolation(corrTimes, betaData)

In [19]:
corr = ql.TenorOptionletVTSTwoParameterCorrelation(rho, beta)

AttributeError: No constructor defined

In [20]:
ql.TenorOptionletVTS()

TypeError: __init__() missing 4 required positional arguments: 'baseVTS', 'baseIndex', 'targIndex', and 'correlation'